In [8]:
import json

# 若需要路徑操作可 import Path，但此處示範直接用檔名即可
# from pathlib import Path

# 若有性質對應表，可在此定義
gender_map = {
    "m sg": "陽性單數",
    "f sg": "陰性單數",
    "n sg": "中性單數",
    "m pl": "陽性複數",
    "f pl": "陰性複數",
    "n pl": "中性複數",
    "pl": "複數"
}

def convert_gender_number(gn):
    """將 'm sg' 等字串轉換為中文語意。若無對應則原樣返回"""
    return gender_map.get(gn, gn)

def convert_case(case):
    """將德文格位轉為中文描述"""
    case_map = {
        "Akkusativ": "第四格",
        "Dativ": "第三格",
        "Genitiv": "第二格"
    }
    return case_map.get(case, case)

def json_to_md(json_file, output_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    md_lines = []

    md_lines.append(f"# **動詞整理**\n\n")

    # 逐一處理 verb_phrases
    for vp in data["verb_phrases"]:
        lemma = vp["lemma"]
        zh_meaning = vp["zh_meaning"]
        
        # 標題：動詞原形
        md_lines.append(f"## {lemma}\n")

        # 中文含義
        md_lines.append(f"- **中文含義**：{zh_meaning}\n")

        # grammar 區塊
        grammar_info = vp.get("grammar", {})
        case_req = grammar_info.get("case_requirement")
        is_reflexive = grammar_info.get("is_reflexive", False)
        is_modal = grammar_info.get("is_modal", False)
        gov_prep = grammar_info.get("gov_preposition")
        is_separable = grammar_info.get("is_separable_verb", False)
        voice = grammar_info.get("voice")  # "Aktiv" or "Passiv"
        mood = grammar_info.get("mood")    # "Indikativ", "Konjunktiv II" etc.

        # 語法說明 (以條列式呈現)
        # md_lines.append("\n### 語法資訊\n")
        
        # 1. case_requirement
        if case_req:
            md_lines.append(f"- **受詞格位需求**：{convert_case(case_req)}\n")
        # 2. 是否為反身動詞
        if is_reflexive:
            md_lines.append("- **特性**：反身動詞\n")
        # 3. 是否為情態動詞
        if is_modal:
            md_lines.append("- **特性**：情態動詞\n")
        # 4. 是否為分離動詞
        if is_separable:
            md_lines.append("- **特性**：分離動詞\n")
        # # 5. 語態
        # if voice:
        #     md_lines.append(f"- **語態**：{voice}\n")
        # # 6. 語氣
        # if mood:
        #     md_lines.append(f"- **語氣**：{mood}\n")
        # 7. 主要支配介詞
        if gov_prep:
            md_lines.append(f"- **主要支配介詞**：{gov_prep}\n")
        
        # prepositional_phrases 若需要詳細處理，可在此擴充
        
        md_lines.append("\n")  # 空行

        # 範例句分析
        examples = vp.get("examples", [])
        if examples:
            md_lines.append("\n### 例句分析\n")
        
        for idx, ex in enumerate(examples, 1):
            md_lines.append(f"{idx}. **德文例句**：`{ex['de']}`\n")
            md_lines.append(f"   - 中文翻譯：{ex['zh']}\n")

            comp = ex.get("components", {})

            # 主語
            subject_data = comp.get("subject")
            if subject_data:
                subj_de = subject_data.get("de", "")
                subj_zh = subject_data.get("zh", "")
                subj_gn = subject_data.get("gender_number", "")
                subj_gn_zh = convert_gender_number(subj_gn) if subj_gn else ""
                
                md_lines.append(f"   - 主語：{subj_de}（{subj_zh}）\n")
                if subj_gn_zh:
                    md_lines.append(f"       * 性質：{subj_gn_zh}\n")

            # 動詞形態
            verb_form = comp.get("verb_form", "")
            md_lines.append(f"   - 動詞形態：{verb_form}\n")

            # 受詞 objects
            objects = comp.get("objects", [])
            for obj in objects:
                obj_de = obj.get("de", "")
                obj_zh = obj.get("zh", "")
                obj_case = obj.get("case", "")
                obj_gn = obj.get("gender_number", "")
                obj_gn_zh = convert_gender_number(obj_gn) if obj_gn else ""
                obj_orig = obj.get("original_nominative_form", "")
                
                md_lines.append(f"   - 受詞：{obj_de}（{obj_zh}, {convert_case(obj_case)}）\n")
                if obj_gn_zh:
                    md_lines.append(f"       * 性質：{obj_gn_zh}\n")
                if obj_orig:
                    md_lines.append(f"       * 原形：{obj_orig}\n")

            # 不定式短語 infinitive_phrases
            inf_phrases = comp.get("infinitive_phrases", [])
            for inf_phr in inf_phrases:
                inf_de = inf_phr.get("de", "")
                inf_zh = inf_phr.get("zh", "")
                inf_comment = inf_phr.get("comment", "")
                md_lines.append(f"   - 不定式短語：{inf_de}（{inf_zh}）\n")
                if inf_comment:
                    md_lines.append(f"       * 說明：{inf_comment}\n")

            # 關係子句 relative_clauses
            rel_clauses = comp.get("relative_clauses", [])
            for rc in rel_clauses:
                rc_de = rc.get("de", "")
                rc_zh = rc.get("zh", "")
                rc_comment = rc.get("comment", "")
                md_lines.append(f"   - 關係子句：{rc_de}（{rc_zh}）\n")
                if rc_comment:
                    md_lines.append(f"       * 說明：{rc_comment}\n")

            # 法律出處
            if ex.get("legal_citations"):
                citations = "；".join(ex["legal_citations"])
                md_lines.append(f"   - 法律出處：{citations}\n")

            md_lines.append("")  # 與下一例句間空行

        # 介詞搭配
        prepositions = vp.get("preposition_combinations", [])
        if prepositions:
            md_lines.append("\n### 介詞搭配\n")
            for prep_info in prepositions:
                prep = prep_info.get("preposition", "")
                prep_case = prep_info.get("case", "")
                ex_de = prep_info.get("example", {}).get("de", "")
                ex_zh = prep_info.get("example", {}).get("zh", "")
                md_lines.append(f"- {prep} + {convert_case(prep_case)}\n")
                if ex_de or ex_zh:
                    md_lines.append(f"  - 例：{ex_de}（{ex_zh}）\n")
        
        md_lines.append("\n***\n")  # 分隔線

    # 元數據輸出
    metadata = data.get("metadata", {})
    md_lines.append(f"> **資料來源**：{metadata.get('source', '不明')}  \n")
    md_lines.append(f"> **語言版本**：{metadata.get('language', '不明')}  \n")
    md_lines.append(f"> **處理時間**：{metadata.get('processed_date', '不明')}  \n")
    md_lines.append(f"> **版本**：{metadata.get('version', '未知')}\n")

    with open(output_file, 'w', encoding='utf-8') as f:
        f.writelines(md_lines)

In [9]:
 

# 使用範例
json_to_md('german_verbs_tw.json', 'output_verbs_tw.md')